In [ ]:
%load_ext nb_black

from collections import Counter

import geopandas as gpd
import pandas as pd
import pyproj
from shapely.ops import transform
from IPython.display import Image

from preprocess.utils import load_supersection_shapes
from project import ARBProject
from primer_graphics import overview_png, norcal_png

from utils import PROJECTED_CRS, load_retro_from_json

import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import cartopy
import cartopy.feature as cf

from geoviews import opts
from cartopy import crs as ccrs

gv.extension("matplotlib", "bokeh")

gv.output(dpi=120, fig="svg")

hv.output(backend="bokeh")


import logging

logging.captureWarnings(True)  # holoviews is a little chatty.

In [ ]:
arb_supersections = load_supersection_shapes()

arb_supersections.geometry = arb_supersections.simplify(
    tolerance=5000
)  # straightens out < 1km wiggles

In [ ]:
cp_lut = pd.read_csv("data/2015_aa_lut.csv")
norcal_aa = cp_lut[cp_lut["supersection"] == "Northern California Coast"]
lst = ["assessment_area", "site_class", "common_practice"]

In [ ]:
retro = load_retro_from_json("data/projects.json")


proj_count = Counter()
for lst in retro["project"]["super_section"].to_dict().values():
    for val in lst:
        proj_count[val] += 1

proj_count = pd.Series(proj_count)
proj_count.name = "n_arb_proj"

arb_supersections = arb_supersections.join(proj_count)

In [ ]:
states = gpd.read_file(
    "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_1_states_provinces.zip"
)
ca = states.loc[states.name == "California"]
ca_ecomap = gpd.overlay(arb_supersections.to_crs(ca.crs), ca, how="intersection")
ca_ecomap["is_northcoast"] = ca_ecomap["SSection"] == "Northern California Coast"

# Supersections, Assessment Areas, and Site Classes 

This notebook will introduce you to key concepts needed to understand how "Improved Forest Management" (IFM) projects are allocated credits. 
IFM projects have a unique feature in that they receive an "upfront" tranche of offset credits (termed ARBOCs, which stands for ARB offset credits). 
Though the precise calculation is somewhat involved, in practice the calculation hinges on two numbers: initial carbon stock (ICS) and common practice (CP). 
This document explains how CP is derived. 


## How is common practice calculated? 
ARB provides a pre-generated look-up table of common practice values. 
Ultimately, common practice links back to per-tree esitmates of carbon storage, as estimated from Forest Inventory and Analysis (FIA) data. 
However, for purposes of gaining a stronger intutition about how IFM projects are assigned common practice, we'll treat the ARB provided common practice values as a given. 

Determining per project common practice requires three pieces of information: 

1. Determine the *geographic* "supersection(s)" where the project falls.
2. Within each supersection, stratify forested land within the project by assessment area (forest type).
3. Within each assessment area, further stratify by "site class" (three choices: low, high, all) 

With these three pieces of information, a project owner or developer can then use the ARB provided common practice lookup table to calculate area weighted common practice. 

## Schematic representation of looking up CP
Graphically, the relationship between the three stages of calculating common practice looks something like this:  

In [ ]:
Image(overview_png())

## A (California) specific example. 
Let's run through a more specific example, using a real supersection, the `Northern California Coast`. 
As its name suggests, the `Northern California Coast` supersection hugs the coastline of northern California, from Marin to the Oregon border (with a small incongruity around Klamath, CA).
A large fraction of all ARB approved IFM projects fall within this supersection and its neighbor, the `Southern Cascades` supersection.


In [ ]:
%%output backend='matplotlib', fig='svg'
gv.Polygons(ca_ecomap, vdims=["is_northcoast"]).opts(cmap="Dark2_r")

### Asssessmnet Area Assignation  
Any ARB IFM project that overlays the supersection area (shown in green) then has to assign its forests to one of the two assessment areas: `Redwood/Douglas-fir Mixed Conifer` or `Oak Woodland`. 


In [ ]:
Image(norcal_png())

A project developer would then need to assign each parcel of their land to one of these two assessment areas.
To do this, ARB provides a list of species that are associated with each assessment area. 
From the assessment area lookup table, we see that `Redwood/Douglas-fir` consists of the following species: 


In [ ]:
norcal_aa.loc[norcal_aa["assessment_area"].str.lower().str.contains("redwood/douglas")][
    "species"
].iloc[0].split(",")

It's important to note that the species list is mostly for guidance.
It appears that developers have a degree of discretion in assigning land to assessment areas. 
The most easy to understand example of this discretion occurs when a project consists of species that do not appear in either any of the supersection's assessment area species lsits. 
That said, these are more technical details that warrant further discussion elsewhere. 

### Site class
The final stage is to assign each parcel of land, within an assessment area, to a site class. 
There are two site class desigations that are mutually exclusive: `high/low` and `all`.
In the case of Northcost `Oak Woodlnd`, there is no site class subdivision, so all parcels assigned to the assesssment area have the same site class and, as a result, the same common practice.
The `Redwood/Douglas-fir` assessment area is an example of a `high/low` assessment area, meaning all `Redwood/Douglas-fir` parcels must be further stratified into high vs low site class (typically based off of soil properties).

## The Lookup
With all these parameters in place, we're finally ready to look up common practice. 
From the ARB provided table, we take supersection, assessment area, and site class to extract the assigned common practice

In [ ]:
norcal_aa[["assessment_area", "site_class", "common_practice"]]